In [1]:
import pandas as pd
import numpy as np
import datetime,os,sys,time
import matplotlib.pyplot as plt
import scipy as sc
import seaborn as sns
import sklearn as sk

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from IPython.display import display
pd.options.display.max_columns = None

# Lets explore the data

In [2]:
def list_files_2(startpath,form):
    gd=[os.path.join(root,f) for root,dirs,files in os.walk(startpath) for f in files if f.endswith(form)]
    return gd

In [3]:
actual_path = os.getcwd()

In [4]:
files = list_files_2(actual_path,'.dat')
files.sort()
types = np.unique([item.split('/')[-2] for item in files])
print('Files available for this study: ')
counter0,counter1,optional,protocol = [0,0,[],[]]
for item in files:
    if types[0] in item:
        counter0 += 1
        if counter0 == 1:
            print('   >> '+types[0])
        print('       ** '+item)
        if types[0] == 'Optional': optional.append(item)
        if types[0] == 'Protocol': protocol.append(item)
    if types[1] in item:
        counter1 += 1
        if counter1 == 1:
            print('   >> '+types[1])
        print('       ** '+item)
        if types[1] == 'Optional': optional.append(item)
        if types[1] == 'Protocol': protocol.append(item)

Files available for this study: 
   >> Optional
       ** /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Optional/subject101.dat
       ** /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Optional/subject105.dat
       ** /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Optional/subject106.dat
       ** /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Optional/subject108.dat
       ** /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Optional/subject109.dat
   >> Protocol
       ** /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Protocol/subject101.dat
       ** /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Protocol/subject102.dat
       ** /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Protocol/subject1

In [ ]:
IMU = ['Temp (°C)','AX','AY','AZ','AX2','AY2','AZ2','Giros1','Giros2','Giros3','MX','MY','MZ','O1','O2','O3','O4']
cols = ['Time (s)','Activity ID','Heart Rate (bpm)']+['Hand '+item for item in IMU]+['Chest '+item for item in IMU]+['Ankle '+item for item in IMU]

In [ ]:
%%time
counter = 0
for item in protocol:
    print('Iteration '+str(counter)+' | File: '+item)
    data = pd.read_csv(item, sep=r'\s{1,}', engine='python', header=None)
    data.columns = cols
    data['User ID'] = int(''.join([e for e in item.split('/')[-1].split('.')[0] if e.isnumeric()]))
    if counter == 0:
        final_data = data
    else:
        final_data = pd.concat([final_data,data])
        final_data = final_data.reset_index(drop=True)
    del data
    counter += 1

Iteration 0 | File: /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Protocol/subject101.dat
Iteration 1 | File: /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Protocol/subject102.dat
Iteration 2 | File: /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Protocol/subject103.dat
Iteration 3 | File: /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Protocol/subject104.dat
Iteration 4 | File: /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Protocol/subject105.dat
Iteration 5 | File: /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Protocol/subject106.dat
Iteration 6 | File: /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Protocol/subject107.dat
Iteration 7 | File: /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/

In [ ]:
%%time
counter = 0
for item in optional:
    print('Iteration '+str(counter)+' | File: '+item)
    data = pd.read_csv(item, sep=r'\s{1,}', engine='python', header=None)
    data.columns = cols
    data['User ID'] = int(''.join([e for e in item.split('/')[-1].split('.')[0] if e.isnumeric()]))
    if counter == 0:
        final_data_optional = data
    else:
        final_data_optional = pd.concat([final_data_optional,data])
        final_data_optional = final_data_optional.reset_index(drop=True)
    del data
    counter += 1

Iteration 0 | File: /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Optional/subject101.dat
Iteration 1 | File: /home/ivan/Desktop/IRONHACK_DATA_ANALYTICS/madrid-oct-2018/final-project/my_project/Optional/subject105.dat


In [ ]:
display(final_data.head())
display(final_data.describe())
display(final_data.dtypes)

In [ ]:
display(final_data_optional.head())
display(final_data_optional.describe())
display(final_data_optional.dtypes)

# First Step > DELETE O1, O2 and O3 Attributes

In [ ]:
final_cols = [e for e in final_data.columns.tolist() if not 'O' in e]
df = final_data[final_cols]
del final_data

final_cols_op = [e for e in final_data_optional.columns.tolist() if not 'O' in e]
df_op = final_data_optional[final_cols_op]
del final_data_optional

# Lets check the missing data by USER ID in % For Protocol Files

In [ ]:
counter = 0
missing = []
for item in df['User ID'].unique():
    a = df[df['User ID'] == item]
    print('User ID: ',item)
    b = a.isna().sum()
    if counter == 0:
        ind = b.index.tolist()
    missing.append(np.round(b.values*100/len(a),2))

In [ ]:
missing_df_user = pd.DataFrame(np.asarray(missing).T,columns=df['User ID'].unique(),index=ind)
display(missing_df_user)

# Lets check the missing values by Activity ID For Protocol Files

In [ ]:
counter = 0
missing = []
act_list = df['Activity ID'].unique()
act_list.sort()
for item in act_list:
    a = df[df['Activity ID'] == item]
    print('Activity ID: ',item)
    b = a.isna().sum()
    if counter == 0:
        ind = b.index.tolist()
    missing.append(np.round(b.values*100/len(a),2))

In [ ]:
missing_df_act = pd.DataFrame(np.asarray(missing).T,columns=act_list,index=ind)
display(missing_df_act)

# NOW FOR OPTIONAL FILES

### USER ID

In [ ]:
counter = 0
missing = []
for item in df_op['User ID'].unique():
    a = df_op[df_op['User ID'] == item]
    print('User ID: ',item)
    b = a.isna().sum()
    if counter == 0:
        ind = b.index.tolist()
    missing.append(np.round(b.values*100/len(a),2))

In [ ]:
missing_df_user_op = pd.DataFrame(np.asarray(missing).T,columns=df_op['User ID'].unique(),index=ind)
display(missing_df_user_op)

### ACTIVITY ID

In [ ]:
counter = 0
missing = []
act_list_op = df_op['Activity ID'].unique()
act_list_op.sort()
for item in act_list_op:
    a = df_op[df_op['Activity ID'] == item]
    print('Activity ID: ',item)
    b = a.isna().sum()
    if counter == 0:
        ind = b.index.tolist()
    missing.append(np.round(b.values*100/len(a),2))

In [ ]:
missing_df_act_op = pd.DataFrame(np.asarray(missing).T,columns=act_list_op,index=ind)
display(missing_df_act_op)

In [ ]:
df_op[df_op['User ID']==101].head()

### Missing values (do not take in count Heart Rate variable)

In [ ]:
counter = 0 
cols1 = [e for e in df.columns.tolist() if e!='Heart Rate (bpm)']
for item in cols1:
    if counter == 0: 
        conditional = df[item].isna()
    else:
        a = conditional | df[item].isna()
        conditional = a
    counter += 1
conditional_1 = conditional
print('Protocol DF >> NaN rows: ',sum(conditional),' | Original rows: ',len(df),' | % of NaN rows: ',round(sum(conditional)*100/len(df),2))

In [ ]:
counter = 0 
cols2 = [e for e in df_op.columns.tolist() if e!='Heart Rate (bpm)']
for item in cols2:
    if counter == 0: 
        conditional = df_op[item].isna()
    else:
        a = conditional | df_op[item].isna()
        conditional = a
    counter += 1
conditional_2 = conditional
print('Optional DF >> NaN rows: ',sum(conditional),' | Original rows: ',len(df_op),' | % of NaN rows: ',round(sum(conditional)*100/len(df_op),2))

In [ ]:
a = df[['Time (s)','Heart Rate (bpm)','User ID']]
list_us = a['User ID'].unique()

for item in list_us:
    b = a[a['User ID']==item].interpolate()
    b = b.interpolate('nearest')
    a.loc[a['User ID']==item,:] = b

In [ ]:
list_us.sort()
counter = 1
fig = plt.figure(figsize=(10,6.5*len(list_us)))
for item in list_us:
    index = int(str(len(list_us))+'1'+str(counter))
    b = a[a['User ID']==item]
    ax = plt.subplot(index)
    ax.plot(b['Time (s)'],b['Heart Rate (bpm)'])
    ax.grid()
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Heart Rate (bpm)')
    ax.set_title('User '+str(item))
    counter += 1
plt.show()

In [ ]:
c = df_op[['Time (s)','Heart Rate (bpm)','User ID']]
list_us = c['User ID'].unique()

for item in list_us:
    d = c[c['User ID']==item].interpolate()
    c.loc[c['User ID']==item,:] = d

In [ ]:
list_us.sort()
counter = 1
fig = plt.figure(figsize=(10,6.5*len(list_us)))
for item in list_us:
    index = int(str(len(list_us))+'1'+str(counter))
    b = c[c['User ID']==item]
    ax = plt.subplot(index)
    ax.plot(b['Time (s)'],b['Heart Rate (bpm)'])
    ax.grid()
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Heart Rate (bpm)')
    ax.set_title('User '+str(item))
    counter += 1
plt.show()

### After Time Variable is interpolated I proceed to drop rows with NaN values. It is supossed to obtain the 99% of the original rows once we have calculated we have less than 1% of rows with NaN values

In [ ]:
df['Heart Rate (bpm)'] = a['Heart Rate (bpm)']
df_op['Heart Rate (bpm)'] = c['Heart Rate (bpm)'] 

Lets check that in `Heart Rate (bpm)` variable we have 0 or less than 1% rows with NaN values

In [ ]:
print()
print('Protocol DF >> N NaN rows for Heart Rate: ',df['Heart Rate (bpm)'].isna().sum(),' | Total rows: ',len(df),' | % of NaN rows: ',round(df['Heart Rate (bpm)'].isna().sum()*100/len(df),4))
print('Protocol DF >> NaN rows: ',sum(conditional_1),' | Original rows: ',len(df),' | % of NaN rows: ',round(sum(conditional_1)*100/len(df),2))
print()
print('Optional DF >> N NaN rows for Heart Rate: ',df_op['Heart Rate (bpm)'].isna().sum(),' | Total rows: ',len(df_op),' | % of NaN rows: ',round(df_op['Heart Rate (bpm)'].isna().sum()*100/len(df_op),4))
print('Optional DF >> NaN rows: ',sum(conditional_2),' | Original rows: ',len(df_op),' | % of NaN rows: ',round(sum(conditional_2)*100/len(df_op),2))
print()

## NICE! Now lets drop NAN rows!

In [ ]:
df_nona = df.dropna()
df_op_nona = df_op.dropna()

In [ ]:
# Checking if we have droped many rows

In [ ]:
print('PROTOCOL DF > Original N of registers: ',len(df),' > After droped the nan rows: ',len(df_nona),' > Rows droped: ',len(df)-len(df_nona),' and in % ',round((len(df)-len(df_nona))*100/len(df),2))
print('OPTIONAL DF > Original N of registers: ',len(df_op),' > After droped the nan rows: ',len(df_op_nona),' > Rows droped: ',len(df_op)-len(df_op_nona),' and in % ',round((len(df_op)-len(df_op_nona))*100/len(df_op),2))

### MORE DROPING 

Note: data labeled with activityID=0 should be discarded in any kind of analysis. This data mainly
covers transient activities between performing different activities, e.g. going from one location to the
next activity's location, or waiting for the preparation of some equipment. Also, different parts of one
subject's recording (in the case when the data collection was aborted for some reason) was put together
during these transient activities (noticeable by some “jumping” in the HR-data).

In [ ]:
print('Protocol DF rows for activity 0: ',len(df_nona[df_nona['Activity ID']==0]),' > Total rows: ',len(df_nona),' in % ',round(len(df_nona[df_nona['Activity ID']==0])*100/len(df_nona),2))
print('Optional DF rows for activity 0: ',' > Total rows: ',len(df_op_nona[df_op_nona['Activity ID']==0]),' > Total rows: ',len(df_op_nona),' in % ',round((len(df_op_nona[df_op_nona['Activity ID']==0])*100/len(df_op_nona)),2))

In [ ]:
df_fin = df_nona[df_nona['Activity ID']!=0].copy()
df_op_fin = df_op_nona[df_op_nona['Activity ID']!=0].copy()

### And Finally lets concatenate OPTIONAL and PROTOCOL dataframes to retrieve the final dataframe to work with

In [ ]:
df_f = pd.concat([df_fin,df_op_fin])
df_f = df_f.reset_index(drop=True)

In [ ]:
# FREE RAM SPACE
del df
del df_fin
del df_nona
del df_op
del df_op_fin
del df_op_nona

### Also we are going to drop USER ID Feature, is not longer needed for further analysis

In [ ]:
df_f.drop('User ID',axis=1,inplace=True)

In [ ]:
print(df_f.columns)
display(df_f.head())

___

## Checking correlations of Features

In [ ]:
k = abs(df_f.corr())
# I filter the data to display only correlations over 0.65 which I stimate as high correlation between Features
k = k[k>0.6]

In [ ]:
plt.figure(figsize=(10,10))

sns.heatmap(k,cmap=plt.cm.plasma)
plt.grid()
plt.title('Correlation Between Features')

plt.show()

In [ ]:
# I have observed that we have correlation between hand and chest temperature. ALso I observe that all acceletartion sensors
# have a high collinearity with the second pair of sensors for each part of the body. Acceleration sensors indexed as 1 (or wihout index)
# are correlated with the sensors indexed as 2. But just with the sensors of he same body place.
# I also have collinearity between Time and Heart Rate Features, between Chest Temp, Hand Temp and Ankle Temp, between Chest magnetic Z axis 
# (Chest MZ), Chest acceleration Z axis sensors 1 and 2 (Chest AZ/AZ2)

### NEXT STEP CONSISTS ON DROPING ALL THESE COLUMNS TO AVOID PROBLEMS WHEN WE TRY TO CLUSTERIZE AND PREDICT

In [ ]:
columns_to_delete = ['Time (s)','Chest Temp (°C)','Chest MZ']+[item for item in df_f.columns if 'AX2' in item or 'AY2' in item or 'AZ2' in item]
print('Features to Drop:')
print(columns_to_delete)

In [ ]:
data = df_f.drop(columns_to_delete,axis=1)
print('Number of columns droped: ',len(columns_to_delete))
print('Final number of columns: ',len(data.columns))
print('Final number of rows: ',len(data))
display(data.head())

### Let's check the correlations again

In [ ]:
k_d = data.corr()
# I filter the data to display only correlations over 0.65 which I stimate as high correlation between Features
#k_d = k_d[k_d>0.6]

In [ ]:
plt.figure(figsize=(11,10))

sns.heatmap(abs(k_d),cmap=plt.cm.pink_r)
plt.grid()
plt.title('Correlation Between Features')

plt.show()

# Perfect! I have managed to delete Features with high correlation

### Now I proceed to delete the older dataframe to save RAM space

In [ ]:
del df_f

### And we divide the data into 2 dataframes, one with the Feature Acivity ID for further analysis and the other wwith all the data except for the Activity ID Feature

In [ ]:
data_activity = data['Activity ID'].copy()
data_final = data[[item for item in data.columns if item!='Activity ID']].copy()
display(data_activity.head())
display(data_final.head())

In [ ]:
# Deleting the data dataframe
del data

# Lets check how our data is grouped

___

# Our next step is to clusterize the data
For this task I have to normalize the data

In [ ]:
from sklearn.cluster import KMeans